In [1]:
import numpy as np
import pandas as pd
import sys, os
from collections import defaultdict
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

sys.path.append('../Shared')
import processing, Model, Visualization


def call_data(base_path, sub_lst, sub_idx):
    data = pd.read_csv(base_path+sub_lst[sub_idx])
    return data

def data_setup(lst_normal, path_normal, window_size, step_size):
    subjects_data = defaultdict(list)
    subjects_label = defaultdict(list)

    for s_idx, filename in enumerate(lst_normal):
        name = filename.lower()
        subject_id = name.split("n")[0]
        data = call_data(path_normal, lst_normal, s_idx).to_numpy()  # (num_samples, num_channels)

        windows = processing.sliding_window(data, window_size, step_size)  # (num_windows, win_len, ch)

        if "standing" in name: label = 0
        elif "gait" in name: label = 1
        elif "sitting" in name: label = 2
        else: label = -1

        labels = np.full(len(windows), label)

        subjects_data[subject_id].append(windows)
        subjects_label[subject_id].append(labels)

    return subjects_data, subjects_label



def get_X_y(X, y):
    all_X, all_y = [], []

    #for subject_id in subjects_data.keys():
    data_list = X
    label_list = y

    for data, labels in zip(data_list, label_list):
        for w, label in zip(data, labels):  # w: (win_len, ch)
            feat = processing.extract_features(w)  # (num_channels*5,)
            all_X.append(feat)
            all_y.append(label)

    all_X, all_y = np.array(all_X), np.array(all_y)   # (N, num_channels*5)

    return all_X, all_y

# Baseline performance - without adaptation

In [ ]:
path_abnormal = "../../../Data/Gait1/Abnormal/"
path_normal = "../../../Data/Gait1/normal/"

lst_abnormal = os.listdir(path_abnormal)
lst_normal = os.listdir(path_normal)

window_size, step_size = 200, 10
num_channels, num_features = 5, 5  # we extracted 5 features per channel

subjects_data, subjects_label = data_setup(lst_normal, path_normal, window_size, step_size)
ACC_all = []

for subject_id in subjects_data.keys():
    print(f"Sub ID {subject_id}")
    X, y = get_X_y(X=subjects_data[subject_id], y=subjects_label[subject_id])
    X = X.reshape(-1, num_channels, num_features, 1)

    model = Model.build_model(input_shape=X.shape[1:], num_classes=len(np.unique(y)))
    history = model.fit(X, y, epochs=50, batch_size=64, verbose=0)
    Sub_acc = []

    for subject_id_target in subjects_data.keys():
        if subject_id == subject_id_target:
            pass
        else:
            X_target, y_target = get_X_y(X=subjects_data[subject_id_target], y=subjects_label[subject_id_target])
            X_target = X_target.reshape(-1, num_channels, num_features, 1)
            loss, acc = model.evaluate(X_target, y_target, verbose=0)

            #Visualization.learning_plot(history)
            print(f"\t Target sub {subject_id_target} acc => {acc*100:.2f}%")
            Sub_acc.append(acc)
    ACC_all.append(Sub_acc)
    print("\n")

In [13]:
#pd.DataFrame(np.array(ACC_all).T, columns=["S10", "S11", "S1", "S2","S3","S4","S5","S6","S7","S8","S9"]).to_csv("./Baseline_inter_subject.csv", index=False)

# Generalization by adding more subjects

In [ ]:
path_abnormal = "../../../Data/Gait1/Abnormal/"
path_normal = "../../../Data/Gait1/normal/"

lst_abnormal = os.listdir(path_abnormal)
lst_normal = os.listdir(path_normal)

window_size, step_size = 200, 10
num_channels, num_features = 5, 5  # we extracted 5 features per channel

subjects_data, subjects_label = data_setup(lst_normal, path_normal, window_size, step_size)
ACC_all = []

for subject_id in subjects_data.keys():
    print(f"Sub ID {subject_id}")
    X_target, y_target = get_X_y(X=subjects_data[subject_id], y=subjects_label[subject_id])
    X_target = X_target.reshape(-1, num_channels, num_features, 1)

    Sub_acc = []
    X_list, y_list = [], []
    for idx, subject_id_target in enumerate(subjects_data.keys()):
        if subject_id == subject_id_target:
            pass
        else:
            X_tmp, y_tmp = get_X_y(X=subjects_data[subject_id_target], y=subjects_label[subject_id_target])
            X_tmp = X_tmp.reshape(-1, num_channels, num_features, 1)

            X_list.append(X_tmp)
            y_list.append(y_tmp)

            X = np.concatenate(X_list, axis=0)
            y = np.concatenate(y_list, axis=0)

            print(f"\t {X.shape}, {y.shape}")
            model = Model.build_model(input_shape=X.shape[1:], num_classes=len(np.unique(y)))
            #history = model.fit(X, y, validation_data=(X_target, y_target), epochs=50, batch_size=64, verbose=0)
            history = model.fit(X, y, epochs=50, batch_size=256, verbose=0)
            loss, acc = model.evaluate(X_target, y_target, verbose=0)
            print(f"\t Target sub {subject_id_target} acc => {acc*100:.2f}%")
            #print(f"\t Target sub {subject_id_target} acc => {np.max(history.history['val_accuracy'])*100:.2f}%")
            Sub_acc.append(acc)

    ACC_all.append(Sub_acc)
    print("\n")

Sub ID 10
	 (3281, 5, 5, 1), (3281,)
	 Target sub 11 acc => 73.30%
	 (6774, 5, 5, 1), (6774,)
	 Target sub 1 acc => 80.12%
	 (10437, 5, 5, 1), (10437,)
	 Target sub 2 acc => 82.21%
	 (13711, 5, 5, 1), (13711,)
	 Target sub 3 acc => 98.19%
	 (17348, 5, 5, 1), (17348,)
	 Target sub 4 acc => 98.71%
	 (20820, 5, 5, 1), (20820,)
	 Target sub 5 acc => 98.50%
	 (25091, 5, 5, 1), (25091,)
	 Target sub 6 acc => 98.22%
	 (29688, 5, 5, 1), (29688,)
	 Target sub 7 acc => 98.22%
	 (34803, 5, 5, 1), (34803,)
